<a href="https://colab.research.google.com/github/LatiefDataVisionary/deep-learning-college-task/blob/main/tasks/week_5_tasks/Task_CNN_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Section 0: Initial Setup (Pengaturan Awal)**

Bagian ini untuk melakukan instalasi library penting yang mungkin belum ada di Colab dan menghubungkan Google Drive.

## **0.1. Install Libraries (Instalasi Library)**

Menginstal library `mtcnn` yang akan digunakan untuk deteksi wajah.

In [15]:
!pip install mtcnn opencv-python Pillow matplotlib pandas numpy scikit-learn tensorflow lz4
!pip install --upgrade Pillow
!pip install lz4

## **0.2. Mount Google Drive (Menghubungkan Google Drive)**

Menghubungkan notebook dengan Google Drive agar dapat mengakses dataset.

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Section 1: Import Libraries and Environment Setup (Impor Library dan Pengaturan Lingkungan)**

**Penjelasan:** Di sini kita akan mengimpor semua modul dan library yang dibutuhkan untuk keseluruhan proyek serta mendefinisikan variabel-variabel global seperti path direktori, ukuran gambar, dan parameter training.

### **1.1. Import Core Libraries (Impor Library Utama)**

**Penjelasan:** Mengimpor library utama seperti tensorflow, keras, numpy, matplotlib.pyplot, os, dan seaborn yang akan digunakan sepanjang proyek ini.

In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
import cv2
import glob
import shutil
from mtcnn.mtcnn import MTCNN

### **1.2. Define Configurations (Definisi Konfigurasi)**

**Penjelasan:** Mendefinisikan variabel-variabel konfigurasi yang akan digunakan di seluruh notebook, termasuk path ke dataset, ukuran gambar yang akan digunakan, ukuran batch untuk training, jumlah epoch, dan jumlah kelas (mahasiswa).

Link Dataset Google Drive: https://drive.google.com/drive/folders/1S5mRxYOfTPAmfqqFFLfbV_D5eWj5J9ox?usp=sharing

In [18]:
# Define Directory Paths (Definisi Path Direktori)
ZIP_PATH = '/content/drive/MyDrive/Dataset/Dataset Sistem Presensi Wajah V1.0.zip' # Path to the raw zip file in Google Drive
RAW_DATA_PATH = '/content/raw_dataset' # Directory to extract the raw dataset
PROCESSED_PATH = '/content/processed_dataset' # Directory to save the processed (face-detected) dataset

# Define Image Parameters (Definisi Parameter Gambar)
IMG_HEIGHT = 128 # Smaller size for custom CNN from scratch
IMG_WIDTH = 128
CHANNELS = 3 # RGB color images

# Define Training Parameters (Definisi Parameter Pelatihan)
BATCH_SIZE = 32
EPOCHS = 50 # Will be controlled by Early Stopping
# NUM_CLASSES will be determined later by the data generator

### **1.3. Extract Dataset (Ekstrak Dataset)**

**Penjelasan:** Mengekstrak file dataset dari Google Drive ke lingkungan Colab agar dapat diakses sebagai direktori biasa.

In [19]:
# Define the path to the zip file in Google Drive
zip_path = '/content/drive/MyDrive/Dataset/Dataset Sistem Presensi Wajah V1.0.zip'
extract_path = '/content/dataset' # Directory to extract the dataset

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Extract the zip file
print(f"Extracting {zip_path} to {extract_path}...")
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Extraction complete.")
except FileNotFoundError:
    print(f"Error: Zip file not found at {zip_path}")
except zipfile.BadZipFile:
    print(f"Error: Could not open or read zip file at {zip_path}. It might be corrupted.")
except Exception as e:
    print(f"An error occurred during extraction: {e}")

# Update TRAIN_DIR and TEST_DIR to point to the extracted directories
# Based on the previous output, the extracted content is in a subfolder
extracted_subfolder = os.path.join(extract_path, 'Dataset Sistem Presensi Wajah V1.0')
TRAIN_DIR = os.path.join(extracted_subfolder, 'Data Train')
TEST_DIR = os.path.join(extracted_subfolder, 'Data Test')


print(f"Updated TRAIN_DIR: {TRAIN_DIR}")
print(f"Updated TEST_DIR: {TEST_DIR}")

# Verify that the directories exist after extraction
if os.path.exists(TRAIN_DIR):
    print(f"TRAIN_DIR exists: {TRAIN_DIR}")
else:
    print(f"Error: TRAIN_DIR not found after extraction at {TRAIN_DIR}")

if os.path.exists(TEST_DIR):
    print(f"TEST_DIR exists: {TEST_DIR}")
else:
    print(f"Error: TEST_DIR not found after extraction at {TEST_DIR}")

# Now it's safe to list contents if needed for verification after extraction
print(f"Contents of {extract_path} after extraction: {os.listdir(extract_path)}")

Extracting /content/drive/MyDrive/Dataset/Dataset Sistem Presensi Wajah V1.0.zip to /content/dataset...
Extraction complete.
Updated TRAIN_DIR: /content/dataset/Dataset Sistem Presensi Wajah V1.0/Data Train
Updated TEST_DIR: /content/dataset/Dataset Sistem Presensi Wajah V1.0/Data Test
TRAIN_DIR exists: /content/dataset/Dataset Sistem Presensi Wajah V1.0/Data Train
TEST_DIR exists: /content/dataset/Dataset Sistem Presensi Wajah V1.0/Data Test
Contents of /content/dataset after extraction: ['Dataset Sistem Presensi Wajah V1.0']


## **Section 2: Advanced Preprocessing - Face Detection and Cropping (Preprocessing Lanjutan - Deteksi dan Pemotongan Wajah)**

**Penjelasan:** Ini adalah tahap paling krusial dan merupakan upgrade utama. Kita akan memproses seluruh dataset mentah sekali jalan. Tujuannya adalah mendeteksi wajah di setiap gambar, memotongnya, dan menyimpannya ke struktur direktori baru yang bersih dan siap pakai. Proses ini menyelesaikan masalah distorsi aspect ratio dan noise latar belakang.

### **2.1. Unzip Raw Dataset (Ekstrak Dataset Mentah)**

**Penjelasan:** Mengekstrak file dataset mentah dari lokasi ZIP_PATH ke direktori RAW_DATA_PATH agar dapat diakses sebagai file gambar.

In [20]:
# Create the raw data extraction directory if it doesn't exist
os.makedirs(RAW_DATA_PATH, exist_ok=True)

# Extract the zip file to the raw data path
print(f"Extracting {ZIP_PATH} to {RAW_DATA_PATH}...")
try:
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(RAW_DATA_PATH)
    print("Extraction complete.")
except FileNotFoundError:
    print(f"Error: Zip file not found at {ZIP_PATH}")
except zipfile.BadZipFile:
    print(f"Error: Could not open or read zip file at {ZIP_PATH}. It might be corrupted.")
except Exception as e:
    print(f"An error occurred during extraction: {e}")

# Verify contents of the extracted raw data directory
print(f"Contents of {RAW_DATA_PATH} after extraction: {os.listdir(RAW_DATA_PATH)}")

# Determine the actual path to the raw image files inside the extracted folder
# Assuming the zip contains a single main folder
extracted_items = os.listdir(RAW_DATA_PATH)
if len(extracted_items) == 1 and os.path.isdir(os.path.join(RAW_DATA_PATH, extracted_items[0])):
    ACTUAL_RAW_DATA_ROOT = os.path.join(RAW_DATA_PATH, extracted_items[0])
else:
    # If structure is different, you might need to adjust this
    ACTUAL_RAW_DATA_ROOT = RAW_DATA_PATH
    print("Warning: Extracted data structure is not a single subfolder. Assuming raw images are directly in RAW_DATA_PATH.")

print(f"Actual root directory for raw images: {ACTUAL_RAW_DATA_ROOT}")

# List a few files to confirm - Search recursively for image files
raw_image_files = glob.glob(os.path.join(ACTUAL_RAW_DATA_ROOT, '**', '*.*'), recursive=True)
# Filter for image files specifically
image_extensions = ['.jpg', '.jpeg', '.png']
raw_image_files = [f for f in raw_image_files if os.path.splitext(f)[1].lower() in image_extensions]


print(f"Found {len(raw_image_files)} raw image files.")
if len(raw_image_files) > 5:
    print("First 5 raw files:", raw_image_files[:5])
elif len(raw_image_files) > 0:
     print("Raw files:", raw_image_files)
else:
    print("No raw image files found. Check ZIP_PATH and extraction process.")

# Update the global variable if needed, though it's already set above
# globals()['ACTUAL_RAW_DATA_ROOT'] = ACTUAL_RAW_DATA_ROOT

Extracting /content/drive/MyDrive/Dataset/Dataset Sistem Presensi Wajah V1.0.zip to /content/raw_dataset...
Extraction complete.
Contents of /content/raw_dataset after extraction: ['Dataset Sistem Presensi Wajah V1.0']
Actual root directory for raw images: /content/raw_dataset/Dataset Sistem Presensi Wajah V1.0
Found 2120 raw image files.
First 5 raw files: ['/content/raw_dataset/Dataset Sistem Presensi Wajah V1.0/Data Test/5231811005_Akhmad Nabil Saputra_13.jpg', '/content/raw_dataset/Dataset Sistem Presensi Wajah V1.0/Data Test/5231911004_al faisal selan 34.jpg', '/content/raw_dataset/Dataset Sistem Presensi Wajah V1.0/Data Test/5231811018_Sulis Septiani Putri_04.jpg', '/content/raw_dataset/Dataset Sistem Presensi Wajah V1.0/Data Test/5231811008_Sophia_39.jpg', '/content/raw_dataset/Dataset Sistem Presensi Wajah V1.0/Data Test/5231911016_Ameliawati_13.jpg']


### **2.2. Initialize Face Detector (Inisialisasi Detektor Wajah)**

**Penjelasan:** Menginisialisasi model MTCNN yang akan digunakan untuk mendeteksi wajah pada setiap gambar.

In [21]:
# Initialize MTCNN detector
detector = MTCNN()
print("MTCNN detector initialized.")

MTCNN detector initialized.


### **2.3. Prepare Processed Directory Structure (Siapkan Struktur Direktori Hasil Proses)**

**Penjelasan:** Membuat struktur direktori baru di PROCESSED_PATH untuk menyimpan gambar wajah yang sudah dideteksi dan dipotong. Struktur ini akan memiliki sub-folder untuk data training dan testing, dan di dalamnya akan ada sub-folder untuk setiap kelas (berdasarkan NIM).

In [22]:
# Clean up and create the processed data directories
if os.path.exists(PROCESSED_PATH):
    print(f"Removing existing processed data directory: {PROCESSED_PATH}")
    shutil.rmtree(PROCESSED_PATH)

os.makedirs(PROCESSED_PATH)
os.makedirs(os.path.join(PROCESSED_PATH, 'train'))
os.makedirs(os.path.join(PROCESSED_PATH, 'test'))
print(f"Created processed data directories: {PROCESSED_PATH}/train and {PROCESSED_PATH}/test")

# Get unique class names (NIMs) from raw filenames by searching recursively
class_names = set()
image_extensions = ['.jpg', '.jpeg', '.png']

# Search recursively for image files within ACTUAL_RAW_DATA_ROOT
all_raw_files_recursive = glob.glob(os.path.join(ACTUAL_RAW_DATA_ROOT, '**', '*.*'), recursive=True)

for filepath in all_raw_files_recursive:
    filename = os.path.basename(filepath)
    if len(filename) >= 10 and os.path.splitext(filename)[1].lower() in image_extensions:
        nim = filename[:10]
        class_names.add(nim)

class_names = sorted(list(class_names))


if not class_names:
    print("Error: No class names (NIMs) extracted from filenames. Check file naming convention and ACTUAL_RAW_DATA_ROOT.")
else:
    print(f"Found {len(class_names)} unique classes (NIMs).")
    # Create sub-folders for each class in train and test directories
    for class_name in class_names:
        os.makedirs(os.path.join(PROCESSED_PATH, 'train', class_name), exist_ok=True)
        os.makedirs(os.path.join(PROCESSED_PATH, 'test', class_name), exist_ok=True)
    print("Created class sub-folders in processed train and test directories.")

# Store class_names for later use
CLASS_NAMES = class_names

Removing existing processed data directory: /content/processed_dataset
Created processed data directories: /content/processed_dataset/train and /content/processed_dataset/test
Found 53 unique classes (NIMs).
Created class sub-folders in processed train and test directories.


### **2.4. Run the Face Detection & Cropping Pipeline (Jalankan Pipeline Deteksi & Pemotongan Wajah)**

**Penjelasan:** Membuat dan menjalankan fungsi untuk mendeteksi wajah di setiap gambar mentah, memotongnya, dan menyimpannya ke struktur direktori yang sudah disiapkan di PROCESSED_PATH. Data akan dibagi secara manual menjadi training dan testing (misal: 80% train, 20% test per kelas).

In [23]:
def process_and_save_faces_by_split(raw_source_dir, processed_dest_dir, detector, img_width, img_height):
    """
    Processes raw images from a specific source directory (e.g., original train or test),
    detects faces, crops them, resizes, and saves to the specified processed
    destination directory based on NIM from filename, maintaining the original split.

    Args:
        raw_source_dir (str): Directory containing raw images for a specific split (e.g., Data Train or Data Test).
        processed_dest_dir (str): Destination directory for processed images of this split (e.g., processed/train or processed/test).
        detector (MTCNN): Initialized MTCNN face detector.
        img_width (int): Target width for processed images.
        img_height (int): Target height for processed images.
    """
    print(f"Starting face detection and cropping for split from {raw_source_dir} to {processed_dest_dir}...")

    image_extensions = ['.jpg', '.jpeg', '.png']
    # Find all image files recursively within the raw source directory
    all_raw_files_recursive = glob.glob(os.path.join(raw_source_dir, '**', '*.*'), recursive=True)
    image_files = [f for f in all_raw_files_recursive if os.path.splitext(f)[1].lower() in image_extensions]

    total_processed = 0
    total_skipped = 0
    files_to_process = len(image_files)

    if files_to_process == 0:
        print(f"No image files found in {raw_source_dir}. Skipping this split.")
        print(f"Face detection and cropping for split finished.")
        print(f"Total images processed and saved: {total_processed}")
        print(f"Total images skipped (no face detected or error): {total_skipped}")
        return

    print(f"Found {files_to_process} images to process in {raw_source_dir}.")


    for i, filepath in enumerate(image_files):
        filename = os.path.basename(filepath)
        if len(filename) >= 10:
            nim = filename[:10]
            # Ensure NIM corresponds to an expected class (from the class_names identified earlier)
            if nim in CLASS_NAMES:
                 # Define the destination path, including the class subfolder
                 dest_class_dir = os.path.join(processed_dest_dir, nim)
                 # Ensure the class directory exists (created in step 2.3)
                 if not os.path.exists(dest_class_dir):
                     # This should not happen if step 2.3 ran correctly, but as a safeguard:
                     os.makedirs(dest_class_dir, exist_ok=True)
                     # print(f"Created missing class directory: {dest_class_dir}") # Uncomment for debugging

                 dest_filepath = os.path.join(dest_class_dir, filename)

                 try:
                    image = cv2.imread(filepath)
                    if image is None:
                        # print(f"Warning: Could not read image file: {filepath}. Skipping.") # Uncomment for detailed skips
                        total_skipped += 1
                        continue

                    # Convert BGR to RGB (MTCNN expects RGB)
                    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                    # Detect faces
                    results = detector.detect_faces(image_rgb)

                    if results:
                        # Get the first detected face (assuming one main face per image)
                        x, y, width, height = results[0]['box']

                        # Add margin (adjust as needed)
                        margin_x = int(width * 0.2)
                        margin_y = int(height * 0.2)
                        x1 = max(0, x - margin_x)
                        y1 = max(0, y - margin_y)
                        x2 = min(image.shape[1], x + width + margin_x)
                        y2 = min(image.shape[0], y + height + margin_y)

                        # Crop the face with margin
                        face_crop = image[y1:y2, x1:x2]

                        # Check if crop is valid (not empty)
                        if face_crop is not None and face_crop.size > 0:
                            # Resize the cropped face to target size
                            face_resized = cv2.resize(face_crop, (img_width, img_height))

                            # Save the processed face image
                            cv2.imwrite(dest_filepath, face_resized)
                            total_processed += 1
                        else:
                             # print(f"Warning: Cropped face is empty for {filepath}. Skipping.") # Uncomment for detailed skips
                             total_skipped += 1
                    else:
                        # print(f"Warning: No face detected in {filepath}. Skipping.") # Uncomment for detailed skips
                        total_skipped += 1

                 except Exception as e:
                    # print(f"Error processing {filepath}: {e}. Skipping.") # Uncomment for detailed errors
                    total_skipped += 1
            else:
                # print(f"Warning: NIM not found in CLASS_NAMES for file {filename}. Skipping.") # Uncomment for debugging unknown NIMs
                total_skipped += 1
        else:
             # print(f"Warning: Filename {filename} is too short to extract NIM. Skipping.") # Uncomment for debugging short filenames
             total_skipped += 1


    print(f"\nFace detection and cropping for split finished.")
    print(f"Total images processed and saved: {total_processed}")
    print(f"Total images skipped (no face detected or error): {total_skipped}")

# --- Run the pipeline for Train and Test splits ---

# Ensure ACTUAL_RAW_DATA_ROOT is correctly determined in step 2.1
if 'ACTUAL_RAW_DATA_ROOT' in globals() and os.path.exists(ACTUAL_RAW_DATA_ROOT):
    original_train_dir = os.path.join(ACTUAL_RAW_DATA_ROOT, 'Data Train')
    original_test_dir = os.path.join(ACTUAL_RAW_DATA_ROOT, 'Data Test')

    # Check if the original train/test directories exist within the extracted raw data
    if not os.path.exists(original_train_dir):
        print(f"Error: Original Train directory not found at {original_train_dir}. Cannot process train data.")
    else:
        process_and_save_faces_by_split(original_train_dir,
                                        os.path.join(PROCESSED_PATH, 'train'),
                                        detector,
                                        IMG_WIDTH, IMG_HEIGHT)

    if not os.path.exists(original_test_dir):
        print(f"Error: Original Test directory not found at {original_test_dir}. Cannot process test data.")
    else:
        process_and_save_faces_by_split(original_test_dir,
                                        os.path.join(PROCESSED_PATH, 'test'),
                                        detector,
                                        IMG_WIDTH, IMG_HEIGHT)

else:
    print("Error: ACTUAL_RAW_DATA_ROOT is not set or does not exist. Cannot run processing pipeline.")

Starting face detection and cropping pipeline from /content/raw_dataset/Dataset Sistem Presensi Wajah V1.0...

Face detection and cropping pipeline finished.
Total images processed and saved: 0
Total images skipped (no face detected or error): 0


### **2.5. Verify Processed Dataset (Verifikasi Dataset Hasil Proses)**

**Penjelasan:** Memeriksa jumlah gambar di direktori training dan testing yang sudah diproses untuk memastikan bahwa pipeline deteksi dan pemotongan wajah berjalan dengan sukses dan data tersimpan dengan benar.

In [24]:
# Function to count images in a directory, including subdirectories
def count_images_in_directory(directory):
    count = 0
    if not os.path.exists(directory):
        return 0
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                count += 1
    return count

# Count images in processed train and test directories
train_count = count_images_in_directory(os.path.join(PROCESSED_PATH, 'train'))
test_count = count_images_in_directory(os.path.join(PROCESSED_PATH, 'test'))

print(f"Number of processed images in training directory ({os.path.join(PROCESSED_PATH, 'train')}): {train_count}")
print(f"Number of processed images in testing directory ({os.path.join(PROCESSED_PATH, 'test')}): {test_count}")

if train_count == 0 or test_count == 0:
    print("Warning: No processed images found in one or both directories. Check the processing pipeline and file paths.")
else:
    print("Processed dataset structure verified.")

Number of processed images in training directory (/content/processed_dataset/train): 0
Number of processed images in testing directory (/content/processed_dataset/test): 0


## **Section 3: Data Loading and Augmentation (Pemuatan dan Augmentasi Data)**

**Penjelasan:** Sekarang kita akan bekerja dengan data yang sudah bersih di PROCESSED_PATH. Karena datanya sudah memiliki struktur folder per kelas (berkat pipeline preprocessing di Section 2), kita bisa menggunakan `ImageDataGenerator` dan metode `flow_from_directory` yang lebih efisien untuk memuat data dan menerapkan augmentasi secara on-the-fly.

### **3.1. Create Data Generators (Membuat Generator Data)**

**Penjelasan:** Menginisialisasi `ImageDataGenerator` untuk data training dengan berbagai teknik augmentasi untuk meningkatkan variasi data dan membantu mencegah overfitting. Satu generator terpisah dibuat untuk data testing yang hanya melakukan rescaling.

In [25]:
# Create ImageDataGenerator for training with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,             # Rescale pixel values to [0, 1]
    rotation_range=20,          # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,      # Randomly shift image horizontally
    height_shift_range=0.2,     # Randomly shift image vertically
    shear_range=0.2,            # Apply shear transformation
    zoom_range=0.2,             # Apply random zoom
    horizontal_flip=True,       # Randomly flip images horizontally
    fill_mode='nearest'         # Fill pixels lost during transformations
)

# Create ImageDataGenerator for testing (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255) # Only rescale for consistency

### **3.2. Apply the Generators (Menerapkan Generator)**

**Penjelasan:** Menggunakan metode `flow_from_directory` dari generator yang telah dibuat untuk membaca gambar langsung dari struktur folder di PROCESSED_PATH/train dan PROCESSED_PATH/test. Ini secara otomatis akan menentukan label kelas berdasarkan nama sub-folder.

In [26]:
# Create training data generator
train_generator = train_datagen.flow_from_directory(
    os.path.join(PROCESSED_PATH, 'train'),
    target_size=(IMG_WIDTH, IMG_HEIGHT), # Target size for input images
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    class_mode='categorical' # For multi-class classification
)

# Create testing data generator
test_generator = test_datagen.flow_from_directory(
    os.path.join(PROCESSED_PATH, 'test'),
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    shuffle=False # Keep data in order for evaluation metrics
)

# Get the number of classes from the training generator
NUM_CLASSES = train_generator.num_classes
print(f"\nNumber of classes (detected from directories): {NUM_CLASSES}")
print(f"Class indices: {train_generator.class_indices}")

# You can also store the class names if needed
CLASS_NAMES_GENERATOR = list(train_generator.class_indices.keys())
print(f"Class names (order corresponds to indices): {CLASS_NAMES_GENERATOR}")

Found 0 images belonging to 53 classes.
Found 0 images belonging to 53 classes.

Number of classes (detected from directories): 53
Class indices: {'5221911012': 0, '5221911025': 1, '5231811002': 2, '5231811004': 3, '5231811005': 4, '5231811006': 5, '5231811007': 6, '5231811008': 7, '5231811009': 8, '5231811010': 9, '5231811013': 10, '5231811014': 11, '5231811015': 12, '5231811016': 13, '5231811017': 14, '5231811018': 15, '5231811019': 16, '5231811021': 17, '5231811022': 18, '5231811023': 19, '5231811024': 20, '5231811025': 21, '5231811026': 22, '5231811027': 23, '5231811028': 24, '5231811029': 25, '5231811030': 26, '5231811031': 27, '5231811033': 28, '5231811034': 29, '5231811035': 30, '5231811036': 31, '5231811037': 32, '5231811038': 33, '5231811039': 34, '5231911001': 35, '5231911002': 36, '5231911003': 37, '5231911004': 38, '5231911005': 39, '5231911006': 40, '5231911007': 41, '5231911008': 42, '5231911009': 43, '5231911010': 44, '5231911011': 45, '5231911012': 46, '5231911013': 47,